In [82]:
import numpy as np
from time import time
from collections import deque
from copy import deepcopy as dcopy
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from queue import PriorityQueue as PQ, LifoQueue as LIFOQ
%matplotlib inline

In [ ]:
"""
TODO LISTA:

DODAWANIE KLUMN DO DF
WYSWIETLANIE / ZAPISYWANIE WYKRESÓW
KOMPARATOR DLA KLASY CHESSBOARD
DODAC LIFOQ dla DFS
DALSZE DEBUGOWANIE

"""

In [2]:
class Chessboard:
    def __init__(self,n = 4, board_state = None, queens_positioned =0): # inicjalizacja
        self.size=n
        self.board = np.zeros((self.size),dtype=np.int) if board_state is None else board_state
        self.queens_counter = queens_positioned
    def __lt__(self, other):
        
    def position(self,row,column): # ustawienie królowej na szachownicy
        self.board[row] = column
        
    def print_board(self,print_full_board=True): # wyświetlenie szachownicy
        if print_full_board:
            full_chessboard = np.zeros((self.size,self.size),dtype=np.int)
            for position, queen_position in zip(range(0,self.size),self.board) :
                full_chessboard [queen_position][position] = 1
            print( full_chessboard, end='\n\n')
            
    def get_positions(self):
        return  self.board[:] , self.queens_counter
    
    def default_position(self,queens_array,positioned):
        self.queens_counter = positioned
        self.board[:]  = queens_array[:]
        
    def add_queen_counter(self):
        self.queens_counter +=1
        
    def get_queen_counter(self):
        return self.queens_counter
    
    def set_queens_counter(self,n_queens):
        self.queens_counter = n_queens

In [3]:
def print_queens_positions(chessboard: Chessboard):
    """
    wyświetl pozycje hetmanów na szachownicy w czytelny dla człowieka sposób
    """
    print(" queens positions: {0}".format(chessboard.get_positions()+1)) # +1 dla czytelności 

In [4]:
def print_stats(n_queens,generate_counter, state_check_counter,correct_counter,timer):
    """
    wyświetl statystyki jednej iteracji eksperymentu
    """
    print("N_queens: {0}".format(n_queens))
    print("Number of correct states: {0}".format(correct_counter))
    print("Number of generated states: {0}".format(generate_counter))
    print("Number of checked states: {0}".format(state_check_counter))
    print("Time spent: {0}".format(timer), end='\n\n')

In [5]:
def build_matrix(board_state,n, queens_palced):
    matrix = np.zeros((n,n),dtype=int)
    for i,y in enumerate(board_state[:queens_palced]):
        matrix[y][i] =1
    return matrix
    

In [6]:
build_matrix([0,2,0,0],4,2) #test

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0]])

In [7]:
#HEURYSTYKI
def H1(board, n_queens, queens_placed): 
    wcol = np.zeros(queens_placed,dtype=np.int)
    for i,row in  enumerate(board[:queens_placed]):
        wcol[i] = (n_queens - row +1 if row <= n_queens / 2 else row)
        #print(wcol[i], row)
    return (n_queens - queens_placed) * np.sum(wcol)

In [8]:
H1(np.array([2,3,0,0]),4,2) # trzeba dodać 1 #test

12

In [9]:
def H2(board,n_queens, queens_placed): 
    matrix = build_matrix(board,n_queens, queens_placed) #tworzymy macierz z wektora hetmanów
    for y,x in  enumerate(board[:queens_placed]):
        # kolumny i wiersze
        matrix[x,:] =1
        matrix[:,y] = 1
        #skosy
        n_elements = min(x,y)
        n_diff = min(n_queens -x-1,y)
        
        diag1_x, diag1_y = x- n_elements, y-n_elements
        diag2_x, diag2_y = x+ n_diff, y-n_diff
        
        max_elements_diag1 = n_queens - max(diag1_x,diag1_y)
        max_elements_diag2 = min(diag2_x, n_queens - diag2_y -1) +1
        for i in range(0,max_elements_diag1):
            matrix[diag1_x+i, diag1_y+i]=1
        for i in range(0,max_elements_diag2):
            matrix[diag2_x-i, diag2_y+i]=1
        #matrix[diag1_x:diag1_x+max_elements_diag1, diag1_y:diag1_y+max_elements_diag1]=1
        #matrix[diag2_x-max_elements_diag2:diag2_x, diag2_x+max_elements_diag2:diag2_x]=1
    print(matrix)
    return matrix.sum(),n_queens**2 - matrix.sum()

In [10]:
H2(np.array([0,2,0,0]),4,2)#test

[[1 1 1 1]
 [1 1 1 0]
 [1 1 1 1]
 [1 1 1 1]]


(15, 1)

In [11]:
def H3(board,n_queens, queens_placed): 
    dh = queens_placed
    S = (n_queens /2) *(n_queens -1)
    counter = Counter(board[:queens_placed]) # dla każdego rzędu
    for values in counter.values():
        if values >1:
            dh -= (values+1)
    return S - dh

In [12]:
H3([0,1,3,0],4,3)#test

3.0

In [54]:
def H(number,n_queens,board, queens_placed):
    switch ={
        0:None,
        1:H1,
        2:H2,
        3:H3
    }
    board_state = board[:] + 1 if number ==1 else board[:]
    heuristic = switch[number]
    if number: return heuristic(board_state,n_queens, queens_placed) 
    else: return 0

In [80]:
class Strategy:
    def __init__(self,size : int, applied_heuristics =0,Brute_Force_enabled = 0,
                 BFS_strategy=True,print_full_board=True, find_first_breakout= True):
        self.state_check_counter = 0 #ilość sprawdzonych stanów przez algorytm
        self.generate_counter =0 # ilość wygenerowanych stanów
        self.correct_counter =0 # ilość odnalezionych rozwiązań
        self.size = size
        #self.q = deque()
        #self.q = []
        self.q =PQ()
        self.bf_enabled = Brute_Force_enabled #true => Brute Force | false =smart
        self.Strategy =BFS_strategy#true =>BFS | false => DFS
        self.print_full_board_bool = print_full_board
        self.find_first = find_first_breakout
        self.applied_heuristics =applied_heuristics
        
    def add_genereted_counter(self):
        self.generate_counter +=1
        
    def add_correct_counter(self):
        self.correct_counter +=1
    
    def add_state_check_counter(self):
        self.state_check_counter +=1
        
    def print_opening(self):
        print('############################################################')
        if self.Strategy and not self.applied_heuristics:
            algo = "Breadth First Search(BFS)" 
        elif   self.Strategy and self.applied_heuristics :
            algo = "Best First Search(BFS)"
        else:
            algo="Depth First Search(DFS)" 
        print("{0} STRATEGY".format(algo))
        print("{0}".format("Brute Force"  if self.bf_enabled else "Inteligent approach" ))
        
        print("{0} Applied".format("Heuristic "+ self.applied_heuristics if self.applied_heuristics else "No Heuristic"))
        
    def append_queue(self, board,heuristic_value =0):
        #self.q.append((heuristic_value, dcopy(board)))
        #self.q=heapq.heappush(self.q, (heuristic_value, dcopy(board)))
        #heapq.heapify(self.q[0][:])
        self.q.put((heuristic_value, dcopy(board)))
        self.add_genereted_counter()
        
    def generate_primary_state(self): # board ma byc obiektem klasy chessboard którym będziemy wysterowywac kolejkę
        board = Chessboard(self.size)
        self.q.put((0,board)) #(heurystyka,obiekt klasy Chessboard)
        
    def check_no_multi_values(self, board_state): #Drugi warunek poprawności stanów
        counter = Counter(board_state) 
        for values in counter.values(): 
             if values > 1: 
                return False
        return True
        
    def inteligent_position(self, board,current_checked_column, n_queen): 
        board_slice = board[:n_queen+1]
        board_slice[n_queen] = current_checked_column
        self.add_state_check_counter()
        if not self.check_no_multi_values(board): return False
        for current_queen in range(0, n_queen):
            if np.abs(current_queen-n_queen) == np.abs(board_slice[current_queen]-current_checked_column):
                return False
        return True
        
    def generate_succesor(self,board): #queen_row => czyli jakiego hetmana będziemy ustawiać       
        primary_state, queen_row = board.get_positions()
        for column_position in range(0, self.size):
            if self.bf_enabled  or self.inteligent_position(primary_state,column_position,queen_row): #prymitywne podejście albo właściwa pozycja bez bicia
                board.position(column_position,queen_row+1)
                board.add_queen_counter()
                heuristic_value = H(self.applied_heuristics,self.size,primary_state, queen_row  )
                self.append_queue(board,heuristic_value)
                board.default_position(primary_state,queen_row)
    
    def check_if_not_attacking(self, board_state):
        for i,h1 in enumerate(board_state):
                for j,h2 in enumerate( board_state):
                    if i != j and np.abs(i-j) ==np.abs(h1-h2)  :
                        return False
        return True
    
    def check_final_board_state(self, board): # test osiągniecia celu
        """
        test ośiągnięcia celu
        """
        print(board.get_positions()) #USUNAC
        if self.bf_enabled: #warunek końcowy Brute force
            if  board.get_queen_counter() < self.size: # jeżeli nie umieściliśmy wszystkich hetmanów zwacamy falsz
                return False
            self.add_state_check_counter()
            #drugi warunek i trzeci >>>> True and True = True 
            return check_no_multi_values(board.get_positions()[0]) and check_if_not_attacking(board.get_positions()[0])
        else: #warunek końcowy smart
            if board.get_queen_counter() ==self.size: return True       
            return False
        
    def pop_board(self):
        #if self.Strategy >0 : #BFS lub BSF
        #    return self.q.popleft()# pobierz z początku listy najstarszy stan
        #else:#STRATEGIA DFS
        #   return  self.q.pop() # pobierz z końca listy ostatnio dodany stan
        #return heapq.heappop(self.q)
        return self.q.get()
    def main_loop(self):
        #while len(self.q): # dopóki lista stanów do przeszukania nie jest pusta
        while not self.q.empty():
            heuristic, oldest_state=self.pop_board() #BOARD_STATE MUSI BYC OBIEKTEM KLASY CHESSBOARD
            print(heuristic, oldest_state.get_positions()) #USUNAC
            n_queens = oldest_state.get_queen_counter()           
            if self.check_final_board_state(oldest_state): # jeżeli został osiągnięty cel
                    print_queens_positions(oldest_state.get_positions()[0]) # wyświetl pierwsze znalezione rozwiązanie
                    oldest_state.get_positions()[0].print_board(self.print_full_board_bool);
                    self.add_correct_counter()
                    if self.find_first: break # jeżeli masz skończyć wyszukiwanie po pierwszym sukcesie 
            if n_queens != self.size:
                self.generate_succesor( oldest_state) # wygeneruj wszystkich potomków aktualnego stanu
        
        return self.generate_counter, self.state_check_counter, self.correct_counter 
    
    

In [19]:
#TODO TRZEBA ZROBIC ZBIERANIE DANYCH I WYSWIETLANIE W WYKRESACH / ZAPISYWANIE WYKRESOW
def data_to_dataframe(df, column_name):
    pass

SyntaxError: unexpected EOF while parsing (<ipython-input-19-20c678d2e9f7>, line 2)

In [ ]:
def plot_scores(df, title):
    pass

In [77]:
def Experiment(n_start=4, n_stop=20, heuristic=0,algo=1,strat= 0,
               print_full_board=True, find_first_breakout= True): #główna pętla eksperymentu
    if n_start < 4 : return 1 # nie istnieją rozwiązania   
    n_array = np.array(list(range(n_start,n_stop+1)))
    generate_list, state_check_list, correct_list, time_list = [],[],[],[]
    for n in n_array:# pętla po wielkości szachownicy 
        timer_start = time()
        strategy = Strategy(n,applied_heuristics =heuristic,Brute_Force_enabled = strat,
                 BFS_strategy=algo,print_full_board=print_full_board, find_first_breakout= find_first_breakout)
        strategy.print_opening()
        strategy.generate_primary_state()
        generate_cnt, state_check_cnt, correct_cnt =strategy.main_loop()
        timer_stop = time()
        timer = timer_stop - timer_start
        generate_list.append(generate_cnt)
        state_check_list.append(state_check_cnt)
        correct_list.append(correct_cnt)
        time_list.append(timer)
        print_stats(n,generate_cnt, state_check_cnt,correct_cnt,timer)
    return generate_list, state_check_list,correct_list,time_list

In [81]:
#experyment dla  BFS
n_start =4
n_stop=20
df_BFS = pd.DataFrame(np.arange(n_start, n_stop),columns=["n_queens"]).set_index('n_queens')
heuristic_list = np.arange(0,4)
bf_enable_list = [1,0]
for h in heuristic_list:
    for bf in bf_enable_list:
        generate_list, state_check_list,correct_list,time_list = Experiment(n_start, n_stop,heuristic=h,algo=1,strat= bf,
                                                                            print_full_board=True, find_first_breakout= True )
        for column in [generate_list, state_check_list,correct_list,time_list]:
            data_to_dataframe(df_BFS,column)
plot_scores(df_BFS, "BFS")

############################################################
Breadth First Search(BFS) STRATEGY
Brute Force
No Heuristic Applied
0 (array([0, 0, 0, 0]), 0)
(array([0, 0, 0, 0]), 0)


TypeError: '<' not supported between instances of 'Chessboard' and 'Chessboard'